# Imports and configuration

**Note:** It is assumed that impatient people do not have a configuration file yet. You can learn on configuration files in the [Setting up a configuration file](Setting up a configuration file.ipynb) notebook. 

Since the parent directory, which contains the `pypmj` module, is not automatically in our path, we need to append it before.

In [ ]:
import sys
sys.path.append('..')
import os
import numpy as np

Specify the **path to your JCMsuite installation directory** here. You can skip this later if you have a configuration file.

In [ ]:
import pypmj as jpy
jpy.import_jcmwave('/path/to/your/JCMsuite/installation/directory')

# Prepare

**Creating a `JCMProject`**: It is assumed that you run this notebook in the a subfolder of the folder structure shipped with pypmj. If this raises an Exception, make sure that the path to the mie2D project is correct.

In [ ]:
project = jpy.JCMProject('../projects/scattering/mie/mie2D')

**Set the keys** that are necessary to translate the JCM template files.

In [ ]:
mie_keys = {'constants' :{}, # <-- can be anything, but is not looped over and not stored in the HDF5 store
            'parameters': {}, # <-- everything that needs to be stored, but is not in layout.jcmt
            'geometry': {'radius':np.linspace(0.3, 0.5, 40)}} # <-- same as before, but layout.jcmt-relevant

**Initialize a `SimulationSet`**. We ignore the configured storage here and save everything in the current working dir.

In [ ]:
simuset = jpy.SimulationSet(project, mie_keys)

**Make a schedule**: this checks which simulations need to be run and sorts them depending on their geometry.

In [ ]:
simuset.make_simulation_schedule()

**Define a processing function**: all post process results are passed to this function automatically. It *must* return a `dict` with your results.

In [ ]:
def read_scs(pp):
    results = {} #must be a dict
    results['SCS'] = pp[0]['ElectromagneticFieldEnergyFlux'][0][0].real
    return results

# Solve

**Is your machine multicore?** Than uncomment this statement and allow yourself some more computation power.

In [ ]:
# simuset.resource_manager.resources['localhost'].set_m_n(4,2)

**Run your simulations**. The results will be appended to the HDF5 store.

In [ ]:
simuset.run(processing_func=read_scs)

**Plot**

In [ ]:
%matplotlib inline
data = simuset.get_store_data().sort_values(by='radius')
data.plot(x='radius', y='SCS', title='Results of the simulation')

**Write the data to CSV**.

In [ ]:
simuset.write_store_data_to_file() # default is results.csv in the storage folder